In [ ]:
pip install requests pandas


# Crawl sách (created_at ở dạng dd/mm/yy)

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime
from google.colab import files

def crawl_tiki_product_ids(category_url, target_reviews=1000):
    """Crawl product IDs từ danh mục Tiki, chỉ lấy sản phẩm có review, cho đến khi đủ số review."""
    product_ids = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    all_reviews = []
    page = 1

    while len(all_reviews) < target_reviews:
        url = f"{category_url}&page={page}"
        try:
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"⚠️ Lỗi khi truy cập trang {page}, mã lỗi: {response.status_code}")
                break

            data = response.json()
            products = data.get("data", [])
            if not products:
                print(f"⚠️ Hết sản phẩm tại trang {page} của danh mục")
                break

            # Lấy product ID có review
            new_product_ids = []
            for product in products:
                product_id = product.get("id")
                review_count = product.get("review_count", 0)
                if product_id and review_count > 0:
                    new_product_ids.append(product_id)
            print(f"Trang {page} | Số product ID có review lấy được: {len(new_product_ids)}")

            # Crawl review cho các product ID mới
            for product_id in new_product_ids:
                print(f"\n📦 Crawl reviews cho sản phẩm ID: {product_id}")
                reviews = get_tiki_reviews(product_id, max_reviews=50)  # Giới hạn 50 review/sản phẩm
                all_reviews.extend(reviews)
                print(f"🌟 Tổng số review hiện tại: {len(all_reviews)}")
                if len(all_reviews) >= target_reviews:
                    break  # Dừng nếu đã đủ review

            product_ids.extend(new_product_ids)
            page += 1
            time.sleep(1)

            # Nếu đã đủ review, dừng
            if len(all_reviews) >= target_reviews:
                print(f"✅ Đã thu thập đủ {len(all_reviews)} review")
                break

        except Exception as e:
            print(f"❌ Lỗi khi crawl trang {page}: {str(e)}")
            break

    return product_ids, all_reviews

def get_tiki_reviews(product_id, max_reviews=50):
    """Lấy review của một sản phẩm, dừng khi không còn review hoặc đạt max_reviews."""
    reviews = []
    page = 1
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    while len(reviews) < max_reviews:
        url = f"https://tiki.vn/api/v2/reviews?product_id={product_id}&page={page}&limit=20&sort=score|desc"
        try:
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"⚠️ Lỗi khi truy cập trang {page} của sản phẩm {product_id}, mã lỗi: {response.status_code}")
                break

            data = response.json()
            items = data.get("data", [])
            if not items:
                print(f"Sản phẩm {product_id} | Hết review tại trang {page}")
                break

            # Lấy số review tối đa còn lại
            remaining_reviews = max_reviews - len(reviews)
            items_to_take = min(len(items), remaining_reviews)
            print(f"Sản phẩm {product_id} | Trang {page} | Số review lấy được: {items_to_take}")

            for item in items[:items_to_take]:
                # Chuyển đổi timestamp sang datetime và định dạng lại
                timestamp = item.get("created_at")
                if timestamp:
                    try:
                        # Chuyển timestamp (giây) thành datetime object
                        dt = datetime.fromtimestamp(timestamp)
                        # Định dạng lại thành dd/mm/yy
                        formatted_date = dt.strftime("%d/%m/%y")
                    except:
                        formatted_date = None
                else:
                    formatted_date = None

                reviews.append({
                    "user_name": item.get("created_by", {}).get("name"),
                    "rating": item.get("rating"),
                    "comment": item.get("content"),
                    "created_at": formatted_date,  # Sử dụng ngày đã định dạng
                    "thank_count": item.get("thank_count"),
                    "product_id": product_id,
                    "images": [img["full_path"] for img in item.get("images", [])]
                })

            if len(reviews) >= max_reviews:
                print(f"Sản phẩm {product_id} | Đã đạt giới hạn {max_reviews} review")
                break

            page += 1
            time.sleep(1)
        except Exception as e:
            print(f"❌ Lỗi khi crawl review sản phẩm {product_id} trang {page}: {str(e)}")
            break

    return reviews

def main():
    # 🟡 Nhập danh mục từ người dùng
    category_input = input("Nhập danh mục (ID danh mục hoặc từ khóa tìm kiếm, ví dụ: '1815' hoặc 'coffee'): ").strip()
    target_reviews = 1000  # Số lượng review tối thiểu

    # Tạo URL dựa trên đầu vào
    if category_input.isdigit():
        # Nếu là ID danh mục
        category_url = f"https://tiki.vn/api/v2/products?category={category_input}&limit=48"
    else:
        # Nếu là từ khóa tìm kiếm
        category_url = f"https://tiki.vn/api/v2/products?q={category_input}&limit=48"

    print(f"🔍 Sử dụng URL: {category_url}")

    # 🔄 Crawl product IDs và reviews
    print("🔍 Bắt đầu crawl từ danh mục...")
    product_ids, all_reviews = crawl_tiki_product_ids(category_url, target_reviews=target_reviews)
    print(f"✅ Tổng số product ID có review thu thập được: {len(product_ids)}")
    print(f"✅ Tổng số review thu thập được: {len(all_reviews)}")

    # 🟢 Chuyển dữ liệu thành DataFrame và lưu file
    if all_reviews:
        df = pd.DataFrame(all_reviews)
        csv_path = f"tiki_reviews_{time.strftime('%Y%m%d_%H%M%S')}.csv"  # Thêm ngày giờ để tránh ghi đè
        try:
            df.to_csv(csv_path, index=False, encoding="utf-8-sig")
            print(f"✅ File CSV đã được lưu: {csv_path}")

            # Tải file về máy
            files.download(csv_path)
            print("📥 Đang tải file CSV về máy...")
        except Exception as e:
            print(f"❌ Lỗi khi lưu hoặc tải file: {str(e)}")
    else:
        print("⚠️ Không có review nào được thu thập.")

if __name__ == "__main__":
    main()

Nhập danh mục (ID danh mục hoặc từ khóa tìm kiếm, ví dụ: '1815' hoặc 'coffee'): 8322
🔍 Sử dụng URL: https://tiki.vn/api/v2/products?category=8322&limit=48
🔍 Bắt đầu crawl từ danh mục...
Trang 1 | Số product ID có review lấy được: 40

📦 Crawl reviews cho sản phẩm ID: 277809376
Sản phẩm 277809376 | Trang 1 | Số review lấy được: 1
Sản phẩm 277809376 | Hết review tại trang 2
🌟 Tổng số review hiện tại: 1

📦 Crawl reviews cho sản phẩm ID: 277748735
Sản phẩm 277748735 | Trang 1 | Số review lấy được: 5
Sản phẩm 277748735 | Hết review tại trang 2
🌟 Tổng số review hiện tại: 6

📦 Crawl reviews cho sản phẩm ID: 277728224
Sản phẩm 277728224 | Trang 1 | Số review lấy được: 20
Sản phẩm 277728224 | Trang 2 | Số review lấy được: 20
Sản phẩm 277728224 | Trang 3 | Số review lấy được: 10
Sản phẩm 277728224 | Đã đạt giới hạn 50 review
🌟 Tổng số review hiện tại: 56

📦 Crawl reviews cho sản phẩm ID: 277607921
Sản phẩm 277607921 | Trang 1 | Số review lấy được: 2
Sản phẩm 277607921 | Hết review tại trang 2
🌟 T

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Đang tải file CSV về máy...
